# Final Model for Paper 1

**IMPORTANT EDIT ON OCTOBER 29, 2025**
_We realized that by doing a clean slate on participants with noisy spectra, we were immensely reducing statistical power. Thus, we decided to openly deal with this noise and keep out spectra for statistical modeling. This is far more robust than imputing and replacing their values with the median of the dataset, especially when we actually HAVE their data_

So we for our final model before LASSO, we skipped `remove_subjects_from_finaldf.py`, but still added the dataset from exgm160 go/no-go for the final analysis. You can see that we are now working with **5830** spectra instead of 5826, as seen in `model_analysis.ipynb`

~~After we ran `model_analysis.ipynb`, there were a handful of participants left within each cluster who had PSD models removed across experiences.~~

~~We want to be able to cleanly analyze data across experiences within each cluster, so that means that the sample size should remain the same, understanding that we would want to look at this through some version of repeated measures statistics.~~

~~`remove_subjects_from_finaldf.py`~~
So we still had  `../results/specparam/final_model/df_final.pkl`, but this time, we had to remove any unique participants identified during the following 3 phases of exclusion:
1. exgm108 and exgm136 because of their failure to follow rules during Go/No-Go
2. Any participant that had at least one PSD removed and listed on `removed_models.csv`
3. Any participant that had at least one PSD listed on `negative_models.csv`
- However, upon further review, all of these participants were already listed on `removed_models.csv`

To review documentation on counts and which clusters will not be analyzed, review the Excel spreadsheet _participants_by_cluster_medication_wide.xlsx_ in the Exergame Google Drive folder in  `! Specparam Analysis/Session 1/final_model/`

**This python script eventually gave us `df_final_cleaned.pkl`**

## `append_additional_spectra.py`
When we ran `df_final_cleaned.pkl` through specparam, we noticed that **exgm169_s1_gonogo** was missing. Upon further review of its preprocessed dataset on Google Drive, we noticed that it was missing IC weights. It was re-preprocessed and individually run through the `SpecParamPrep.m` script to produce spectra that were missing for the final model. So we used this python script to append the .mat file of pwelch spectra associated with this dataset to the existing `df_final_cleaned.pkl` file.

~~This python script eventually gave us `df_final_cleaned_with_additional_spectra.pkl`~~
**UPDATE: The final version works with `df_final_with_additional_spectra`**

## UPDATED Analysis Plan
Throughout the notebook, we complete the following tasks:
1. Fit group spectra
2. Create a .csv export for the following metrics:
    - All peak metrics (4 columns: cfs, pws, bws, model index)
        - Make sure you assign frequency bands based off of cfs value
    - All Model Metrics
        - Aperiodic parameters (2 columns: offsets, exps [in order models])
        - Errors
        - R-squared
    - **Make sure you use the model index to pull relevant information from `df_final`**
3. Plot mean spectra with standard deviation for all 11 clusters. (although we will only be using a select few).
    - Plot with aperiodic fit as dotted line (just for record keeping)
    - Plot as flattened spectrum
        - For these spectra, we will need to plot all participants averaged. However, we will create the following splits:
                - Baseline
                - All Executive Function
                - Shoulder Spectra averaged and Tandem Spectra Average (all participants should have data from all trials)

In [1]:
# Spectral parameterization imports
from specparam import SpectralGroupModel

# Import custom function
from create_specparam_plots import *

### Load Datasets

In [2]:
df = pd.read_pickle('../results/specparam/final_model/df_final_with_additional_spectra.pkl')

### Fit Spectra

In [3]:
# Extract spectra from final dataframe
spectra = np.array([spec for spec in df['spectra']])
freqs = np.arange(251)
# Initialize and fit new SpectralGroupModel on cleaned data
fg = SpectralGroupModel(peak_width_limits=[2, 8], min_peak_height=0.2, peak_threshold=2,
                               max_n_peaks=6, verbose=False)
freq_range = [1, 55]
fg.fit(freqs, spectra, freq_range)
fg.print_results()
fg.save_report('../results/specparam/final_model/final_groupModel_for_Paper1.pdf')

                                                                                                  
                                          GROUP RESULTS                                           
                                                                                                  
                            Number of power spectra in the Group: 5831                            
                                                                                                  
                        The model was run on the frequency range 1 - 55 Hz                        
                                 Frequency Resolution is 1.00 Hz                                  
                                                                                                  
                              Power spectra were fit without a knee.                              
                                                                                                  
          

### Export Summary Data as .csv files
They will be analyzed further to summarize our data for the manuscript

#### Extracting Parameters

In [4]:
# Extract peak parameters
peaks = fg.get_params('peak_params')  # 4 columns: cfs, pws, bws, model index
aps = fg.get_params('aperiodic_params')  # 2 columns: offsets, exps [in order of channels/models]
# Extract goodness-of-fit metrics
errors = fg.get_params('error')
r2s = fg.get_params('r_squared')

##### Peaks Export
For each peak that we export, we will need to export its model's following values from `df`:
1. subject
2. session
3. experience
4. component
5. cluster

In [5]:
peaks_export = pd.DataFrame(peaks, columns=['cf', 'pw', 'bw', 'model_ind'])

# Convert Model Indices to integers to index from df and MERGE
peaks_export['model_ind'] = peaks_export['model_ind'].astype(int)
peaks_merged = peaks_export.merge(
    df[['subject', 'session', 'experience', 'component', 'cluster']],
    left_on='model_ind',
    right_index=True,
    how='left'
)
print("Method 1 - Using pandas merge:")
print(peaks_merged.head())

# Creating a column for frequency bands
def assign_freq_band(cf_value):
    if 1 <= cf_value < 4:
        return "delta"
    elif 4 <= cf_value < 8:
        return "theta"
    elif 8 <= cf_value < 12:
        return "alpha"
    elif 12 <= cf_value < 20:
        return "low_beta"
    elif 20 <= cf_value < 30:
        return "high_beta"
    elif 30 <= cf_value < 55:
        return "gamma"
    else:
        return None  # Default if cf doesn't fall within the specified ranges
peaks_merged['freq_band'] = peaks_merged['cf'].apply(assign_freq_band)

# Save as .csv file for analysis on R
peaks_merged.to_csv('../results/specparam/final_model/peaks.csv', index=False)

Method 1 - Using pandas merge:
          cf        pw        bw  model_ind  subject session     experience  \
0  19.673625  0.382657  8.000000          0  exgm002      s1  digitbackward   
1  10.512218  0.267552  3.274746          1  exgm002      s1   digitforward   
2  19.360175  0.496107  8.000000          1  exgm002      s1   digitforward   
3  10.031609  0.710193  4.141904          2  exgm002      s1         gonogo   
4  18.723401  0.670072  8.000000          2  exgm002      s1         gonogo   

   component  cluster  
0          5       10  
1          5       10  
2          5       10  
3          5       10  
4          5       10  


##### Model Metrics Export
Just like above we will need to export each model's following values from `df`:
1. subject
2. session
3. experience
4. component
5. cluster

In [6]:
# Ensure all arrays are 2D
r2s = r2s.reshape(-1, 1)  # Reshape r2s to (12, 1)
errors = errors.reshape(-1, 1)  # Reshape errors to (12, 1)

# Horizontally stack aperiodic params, R^2, and errors
model_metrics = pd.DataFrame(
    data=np.hstack([aps, r2s, errors]),  # Horizontally stack arrays
    columns=["offset", "exp", "r2", "error"]  # Column names
)

# Add Model Index for set up for joining with model information
model_metrics["model_ind"] = np.arange(len(df), dtype=int)

# Add Model Details to model_metric dataframe
model_metrics_merged = model_metrics.merge(
    df[['subject', 'session', 'experience', 'component', 'cluster']],
    left_on='model_ind',
    right_index=True,
    how='left'
)

# Write .csv to results folder
model_metrics_merged.to_csv('../results/specparam/final_model/model_metrics.csv', index=False)

## Creating Plots for each cluster

In [7]:
create_all_plots(df, fg) # make sure you edit this function by the time you are analyzing two sessions!

Starting plot generation...
Clusters: [np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13)]
Sessions: ['s1']

Processing Cluster 3, Session s1:
  Created baseline plots
  Created cognitive plots
  Created motor plots

Processing Cluster 4, Session s1:
  Created baseline plots
  Created cognitive plots
  Created motor plots

Processing Cluster 5, Session s1:
  Created baseline plots
  Created cognitive plots
  Created motor plots

Processing Cluster 6, Session s1:
  Created baseline plots
  Created cognitive plots
  Created motor plots

Processing Cluster 7, Session s1:
  Created baseline plots
  Created cognitive plots
  Created motor plots

Processing Cluster 8, Session s1:
  Created baseline plots
  Created cognitive plots
  Created motor plots

Processing Cluster 9, Session s1:
  Created baseline plots
  Created cognitive plots
  Created motor plots

Processing Cluster 10, Session s1:
  Cre